In [ ]:
import os
import copy
import glob
import json

import numpy as np
import torch

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pgf import FigureCanvasPgf

mpl.backend_bases.register_backend("pdf", FigureCanvasPgf)
mpl.use("pgf")
%matplotlib inline
%config InlineBackend.figure_format="retina"

# Matplotlib plotting options
plt.style.use("seaborn-paper")
plt.rcParams.update({
    "axes.labelsize": 22,               # label size (x- and y-axis)
    "axes.titlesize": 26,               # title size on (most) axes
    "figure.titlesize": 26,             # title size for fig.sup_title()
    "legend.fontsize": "x-large",       # font size for legend
    "lines.markersize": 6,              # marker size for points and lines
    "lines.markeredgewidth": 2,         # marker edgewidth for points
    "xtick.labelsize": 18,              # label size for x-ticks 
    "ytick.labelsize": 18,              # label size for y-ticks

    "font.family": "serif",             # use serif/main font for text elements
    "text.usetex": True,                # use inline math for ticks
    "pgf.rcfonts": False,               # don't setup fonts from rc params
    "pgf.preamble": [
        # Syling
        r"\usepackage{color}",          # special colors
        r"\setmainfont{DejaVu Serif}",  # serif font via preamble
        r"\usepackage{bm}",             # bold text

        # Math
        r"\usepackage{xfrac}",          # side fractions
        r"\usepackage{amsthm}",         # theorems
        r"\usepackage{amsmath}",        # misc math
        r"\usepackage{amssymb}",        # blackboard math symbols
        r"\usepackage{mathtools}",      # enhance the appearance of math
    ],
})

from profit.models.torch import SequenceOracle, SequenceGPR
from profit.utils.data_utils.tokenizers import AminoAcidTokenizer
from profit.utils.data_utils.serializers import LMDBSerializer

# Model comparison

## Load dataset

In [ ]:
def bbox_to_rect(bbox, color):
    # Convert to matplotlib format: ((upper-left x, upper-left y), width, height).
    return plt.Rectangle(xy=(bbox[0], bbox[1]), width=bbox[2]-bbox[0],
                             height=bbox[3]-bbox[1], fill=False,
                             edgecolor=color, linewidth=2)

In [ ]:
# Preprocess + load the dataset
dataset = LMDBSerializer.load("../../data/3gb1/processed/lstm_fitness/primary_encoding=aa20.mdb")
Xaa = dataset[:]["arr_0"].long()
_labels = dataset[:]["arr_1"].view(-1)

# Determine vocab and sequence length
tokenizer = AminoAcidTokenizer("aa20")
vocab_size = tokenizer.vocab_size
seqlen = Xaa.size(1)

# Convert to onehot
X = torch.zeros(*Xaa.size(), vocab_size)
X.scatter_(2, torch.unsqueeze(Xaa, 2), 1)
X.size()

## Oracle

In [ ]:
paths = sorted(glob.glob("../../bin/3gb1/oracle/*/E*"))
metadata = ["all", "all-weighted", "g-mean", "l-mean", "g-median", "l-median", "g-zero"]
oracle_paths = dict(zip(metadata, paths))
oracle_paths

In [ ]:
# Initialize and load weights for all oracles
oracle_stump = SequenceOracle(seqlen, vocab_size, hidden_size=50, out_size=2)
all_oracles = {}
for desc, path in oracle_paths.items():
    oracle = copy.deepcopy(oracle_stump)
    oracle.load_state_dict(torch.load(path))
    all_oracles[desc] = oracle

# Evaluate on oracle to get predictions
preds = {}
for desc, oracle in all_oracles.items():
    oracle.eval()
    with torch.no_grad():
        oracle_pred = oracle(X)[:,0]
        preds[desc] = oracle_pred
        
# Print topk oracle (pred) values 
for desc, pred in preds.items():
    idx = torch.sort(pred).indices[-10:]
    print(desc.upper(), pred[idx], _labels[idx])

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
xy = np.linspace(_labels.min(), _labels.max()) # x = y
colors = cm.get_cmap("coolwarm")(np.linspace(0, 1, len(preds)))

# First graph
handles = []
for (desc, pred), c in zip(preds.items(), colors):
    h = ax1.scatter(_labels, pred, color=c, edgecolors="black", label=desc)
    handles.append(h)
ax1.plot(xy, xy, "--", c="black")
ax1.tick_params("both")
# ax1.set_xlabel("Ground Truth $y$")
ax1.set_ylabel("Mean Oracle Prediction $\hat{y}$")
ax1.legend(handles, list(preds.keys()), scatterpoints=1)

# Add bounding box (view of 2nd graph)
bbox = np.array([-0.1, 2.75, 2, -0.2])
ax1.add_patch(bbox_to_rect(bbox, 'black'))

# Second graph
for (desc, pred), c in zip(preds.items(), colors):
    ax2.scatter(_labels, pred, color=c, edgecolors="black", label=desc)
ax2.plot(xy, xy, "--", c="black")
ax2.tick_params("both")
# ax2.set_xlabel("Ground Truth $y$")
ax2.legend(handles, list(preds.keys()), scatterpoints=1)
ax2.set_xlim(bbox[[0,2]])
ax2.set_ylim(bbox[[1,-1]][::-1])

plt.tight_layout()
# fig.suptitle("Oracle comparison")
plt.subplots_adjust(top=0.9)
plt.show()

What we can see is that oracle predictions are very poor, when compared to the true GT. In fact, for most sequences, the oracle predicts approximately the same values (with some small variance). This is true even when we train the oracle on a subset of the dataset.

## GPR

In [ ]:
paths = sorted(glob.glob("../../bin/3gb1/gpr/*"))
metadata = ["all", "g-mean", "l-mean", "g-median", "l-median", "g-zero"]
gpr_paths = dict(zip(metadata, paths))
gpr_paths

In [ ]:
# Initialize and load weights for all gprs
all_gps = {desc: torch.load(path) for desc, path in gpr_paths.items()}

# Evaluate on oracle to get predictions
preds = {}
for desc, gp in all_gps.items():
    y_mean = gp.predict(Xaa, return_std=False)
    preds[desc] = y_mean

# Print topk oracle (pred) values
for desc, pred in preds.items():
    pred = pred.squeeze() # flatten array
    idx = torch.sort(pred).indices[-10:]
    print(desc.upper(), pred[idx], _labels[idx])

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
xy = np.linspace(_labels.min(), _labels.max()) # x = y
colors = cm.get_cmap("coolwarm")(np.linspace(0, 1, len(preds)))

# First graph
handles = []
for (desc, pred), c in zip(preds.items(), colors):
    h = ax1.scatter(_labels, pred, color=c, edgecolors="black", label=desc)
    handles.append(h)
ax1.plot(xy, xy, "--", c="black")
ax1.tick_params("both")
ax1.set_xlabel("Ground Truth $y$")
ax1.set_ylabel("Mean Oracle Prediction $\hat{y}$")
ax1.legend(handles, list(preds.keys()), scatterpoints=1)

# Add bounding box (zoomed in view of 2nd graph)
bbox = np.array([-0.02, 0.25, 0.25, -0.2])
ax1.add_patch(bbox_to_rect(bbox, 'black'))

# Second graph
for (desc, pred), c in zip(preds.items(), colors):
    ax2.scatter(_labels, pred, color=c, edgecolors="black", label=desc)
ax2.plot(xy, xy, "--", c="black")
ax2.tick_params("both")
ax2.set_xlabel("Ground Truth $y$")
ax2.legend(handles, list(preds.keys()), scatterpoints=1)
ax2.set_xlim(bbox[[0,2]])
ax2.set_ylim(bbox[[1,-1]][::-1])

plt.tight_layout()
# fig.suptitle("Training data")
plt.subplots_adjust(top=0.9)
plt.show()

What we can see is that predictions are good, but often a little lower than true GT. This might be because a lot of sequences in the original dataset are 0. Even if we train the kernel on the sequences that are greater than 0, the performance is still sub-par at best.

# Condition by Adaptive Sampling (CbAS)

## Select optimal oracle/gp

We want to select the oracle and GT in such a fashion that (a) the oracle values are somewhat improving with each iteration and (b) the GT is not falling substantially (aka being led astray into "poor" regions of the search space) with each iteration. Although not strictly required, it is also important to select the oracle/GT such that the initial training dataset used to train both is the same.

What do we do if the oracle is particularly bad at determining the fitness values between good and bad sequences? This is what is currently happening (see above).

In [ ]:
cbas_files = sorted(glob.glob("../../dumps/3gb1/cbas/*.json"))
cbas_files[:5]

In [ ]:
colors = cm.get_cmap("coolwarm")(np.linspace(0, 1, len(cbas_files)))
plt.figure(figsize=(12,8))

for file, c in zip(cbas_files, colors):
    with open(file, "rb") as json_file:
        values = json.load(json_file)
        # Compute mean and std of the samples for each experiment
        oracle_mean = np.mean(np.ma.masked_invalid(values["y_oracle"]), axis=-1)
        oracle_std = np.std(np.ma.masked_invalid(values["y_oracle"]), axis=-1)
        gt_mean = np.mean(np.ma.masked_invalid(values["y_gt"]), axis=-1)
        gt_std = np.mean(np.ma.masked_invalid(values["y_gt"]), axis=-1)
        # Plot results (error bars for last iteration)
        plt.scatter(oracle_mean, gt_mean, color=c, edgecolors="black")
        plt.errorbar(oracle_mean[-1], gt_mean[-1], xerr=oracle_std[-1], yerr=oracle_std[-1],
                     color=c, fmt='none')

plt.xlabel(r"$y_{\mathrm{oracle(s)}}$")
plt.ylabel(r"$y_{\mathrm{gt}}$")

Generally, we see that as number of iterations increase, both the GT and the oracle predictions (for the samples in that sequence) increases until the oracle reaches its asymptotic upper limit. This means that we have likely converge onto a decent sequence candidate set of sequences (within each iteration) which are not contained in the original dataset. Note that the GT values is largely ignored (still useful to determine if the model is not being led into "bad" sequence regions of the search space).

We want to plot the trajectory of the top sequence (for both the oracle and the GT) across each of the 42 different runs. This will help us determine which is the "best" subset of the dataset to train the model(s) on. For instance, if the GT does not decrease substantially while the oracle is predicting high values that means the dataset is doing good.

In [ ]:
quantile = 0.80
sort = True

for oracle_key in oracle_paths.keys():
    # Gather values (for all GPs) for specified oracle
    plt.figure(figsize=(12,8))
    filepaths = [file for file in cbas_files if f"oracle={oracle_key}__" in file]
    gp_keys = [os.path.split(file)[-1].split(".")[0].split("__")[-1] for file in filepaths]
    colors = cm.get_cmap("coolwarm")(np.linspace(0, 1, len(filepaths)))
    for file, gp_key, c in zip(filepaths, gp_keys, colors):
        with open(file, "rb") as json_file:
            values = json.load(json_file)
        # Compute 80th percentile range for y_oracle
        oracle_samples = np.ma.masked_invalid(values["y_oracle"])
        per = np.percentile(oracle_samples, quantile*100, axis=-1)
        per = np.reshape(per, newshape=(oracle_samples.shape[0], 1))
        oracle_idxs = np.where(oracle_samples > per)
        
        # Average values for samples that are greater than quantile
        oracle_vals = np.zeros_like(oracle_samples)
        oracle_vals[oracle_idxs] = oracle_samples[oracle_idxs]
        oracle_vals = oracle_vals.sum(1) / (oracle_vals != 0).sum(1)
        
        gt_samples = np.ma.masked_invalid(values["y_gt"])
        gt_vals = np.zeros_like(gt_samples)
        gt_vals[oracle_idxs] = gt_samples[oracle_idxs]
        gt_vals = gt_vals.sum(1) / (gt_vals != 0).sum(1)
        
        # Sort oracle values (ascending) when plotting
        if sort:
            oracle_idxs = np.argsort(oracle_vals)
            oracle_vals = oracle_vals[oracle_idxs]
            gt_vals = gt_vals[oracle_idxs]
        
        # Plot values
        plt.plot(np.arange(1, len(oracle_vals)+1), oracle_vals, color=c, linestyle="dashed")
        plt.plot(np.arange(1, len(gt_vals)+1), gt_vals, color=c, label=gp_key)
    plt.xlabel("Timestep ($t$)")
    plt.ylabel(f"${quantile * 100:.0f}\%$ Fitness ($y$)")
    plt.title(f"oracle={oracle_key}")
    plt.legend()
    plt.show()

## Topk (k=10) sequences

From the above trajectories, it seems that no matter the oracle used, the GP trained on indices greater than the mean, seems to perform well in that it does not explore "poor" regions of the sequence-space (as evidence by the fact that the GP remains around its max value). As for the oracle, the best seems to be g-mean/g-median/g-zero as well (since they produce high GT values). Should we compare the topk sequences from each of these runs to see if they concur on similar sequences?

In [ ]:
with open("../../dumps/3gb1/cbas/oracle=g-mean__gp=g-mean.json", "rb") as json_file:
    cbas_results = json.load(json_file)

# What are the topk sequences and their predicted oracle/GT values
oracles = np.array(cbas_results["y_oracle"]).flatten()
topk_idx = np.argsort(oracles)[-10:]

seq_topk = np.array(cbas_results["seq"]).flatten()[topk_idx]
yt_topk = oracles[topk_idx]
yt_gt_topk = np.array(cbas_results["y_gt"]).flatten()[topk_idx]

print(yt_topk, np.mean(yt_topk))
print(yt_gt_topk, np.mean(yt_gt_topk))
print(seq_topk)

In [ ]:
with open("../../dumps/3gb1/cbas/oracle=all-weighted__gp=all.json", "rb") as json_file:
    cbas_results = json.load(json_file)

# What are the topk sequences and their predicted oracle/GT values
oracles = np.array(cbas_results["y_oracle"]).flatten()
topk_idx = np.argsort(oracles)[-10:]

seq_topk = np.array(cbas_results["seq"]).flatten()[topk_idx]
yt_topk = oracles[topk_idx]
yt_gt_topk = np.array(cbas_results["y_gt"]).flatten()[topk_idx]

print(yt_topk)
print(yt_gt_topk)
print(seq_topk)

In [ ]:
with open("../../dumps/3gb1/cbas/oracle=g-mean__gp=all.json", "rb") as json_file:
    cbas_results = json.load(json_file)

# What are the topk sequences and their predicted oracle/GT values
oracles = np.array(cbas_results["y_oracle"]).flatten()
topk_idx = np.argsort(oracles)[-10:]

seq_topk = np.array(cbas_results["seq"]).flatten()[topk_idx]
yt_topk = oracles[topk_idx]
yt_gt_topk = np.array(cbas_results["y_gt"]).flatten()[topk_idx]

print(yt_topk)
print(yt_gt_topk)
print(seq_topk)

# Mutation heatmap

In [ ]:
import pandas as pd
import seaborn as sns

from profit.dataset import generator
from profit.dataset.preprocessors import LSTMPreprocessor

In [ ]:
positions = np.arange(20, 56)
num_mutation_sites = len(positions)
seqs = generator.gen(n=1)

In [ ]:
# Permute each position with amino acid
preprocessor = LSTMPreprocessor(vocab="aa20")
all_seqs = []
for pos in positions:
    for seq in seqs:
        template = list("MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE")
        template[pos] = seq
        all_seqs.append(preprocessor.get_input_feats(template))
all_seqs = torch.Tensor(all_seqs).long()

# Convert to onehot
S = torch.zeros(*all_seqs.size(), vocab_size)
S.scatter_(2, torch.unsqueeze(all_seqs, dim=2), 1)

# Evaluate on (g-mean) oracle to get predictions
gmean_oracle = all_oracles["g-zero"]
if isinstance(gmean_oracle, SequenceGPR):
    mu_oracle, sigma_oracle = gmean_oracle.predict(all_seqs, return_std=True)
    mu_oracle, sigma_oracle = mean_oracle.squeeze(), sigma_oracle.squeeze()
else:
    gmean_oracle.eval()
    with torch.no_grad():
        y_pred = gmean_oracle(S)
        mu_oracle, sigma_oracle = y_pred[:,0], torch.sqrt(y_pred[:,1])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(mu_oracle, label="$\mu(x)$")
plt.fill(np.concatenate([torch.arange(sigma_oracle.size(0)),
                         torch.arange(sigma_oracle.size(0), 0, -1)]),
         np.concatenate([mu_oracle - 1.96 * sigma_oracle,
                         torch.flip(mu_oracle + 1.96 * sigma_oracle, dims=(0,))]),
         alpha=0.3, fc="b", ec="None", label=r"95\% CI")
plt.xlabel("Sequence ($x$)")
plt.ylabel("Fitness ($y$)")
plt.legend()
plt.title("Protein fitness (PDB: 3GB1)")

In [ ]:
mu_oracle = mu_oracle.view(num_mutation_sites, vocab_size)
# # TODO: Impute WT sequences to have fitness=1???
# means[torch.floor(wt_pos_idx).long(), torch.floor(wt_aa_idx).long()] = 1.
# sns.heatmap(means, square=True, cmap="coolwarm")

In [ ]:
topk = 5
max_pos = torch.max(mu_oracle, dim=-1) # max value per position
# Sort in descending order so numbers can be displayed properly
topk_pos_idx = torch.sort(max_pos.values, descending=True).indices[:topk]
topk_aa_idx = max_pos.indices[topk_pos_idx]
topk_vals = max_pos.values[topk_pos_idx]

# Create (numbered) annotation labels
topk_labels = torch.zeros_like(mu_oracle).long()
topk_labels[topk_pos_idx, topk_aa_idx] = torch.arange(1, topk_pos_idx.size(0)+1)

# Create (wildtype) annotation labels
template = list("MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE")
Xaa_wt = torch.Tensor(preprocessor.get_input_feats(template))
wt_pos_idx = torch.arange(len(positions)).float()
wt_aa_idx = Xaa_wt[positions]
# Workaround: adjust positions so that scatter points are centered on heatmap 
wt_pos_idx += 0.5
wt_aa_idx += 0.5

In [ ]:
# NOTE: we add 1 to index since python index starts at 0
df = pd.DataFrame(mu_oracle.numpy(), index=positions+1, columns=seqs).rename_axis(
    "VP Position", axis="index").rename_axis("Amino Acid", axis="columns")
df.head()

In [ ]:
plt.figure(figsize=(12,12))
# Plot topk labels on top of other mutations so that numbers can be displayed
ax = sns.heatmap(df, cmap="coolwarm", square=True)
ax = sns.heatmap(df, cmap="coolwarm", square=True, annot=topk_labels, fmt="d",
            mask=(topk_labels==0).numpy(), cbar=False)
# Highlight topk sequences (w/ bbox)
for pos_idx, aa_idx in zip(topk_pos_idx.numpy(), topk_aa_idx.numpy()):
    ax.add_patch(plt.Rectangle(xy=(aa_idx, pos_idx), width=1, height=1,
                               fill=False, edgecolor='black', lw=1.0))
# Denote WT amino acid (w/ points)
plt.scatter(wt_aa_idx, wt_pos_idx, marker='o', color='black')

So the positions 39,40,53 are the most informative since most amino acid substitutions result in a higher fitness score than the rest. It is, however, important to note that this only takes into account 1 mutation position. In fact, as the number of mutation sites increase, the effects of multi-site mutations no longer become additive, but rather get closer and closer to the WT fitness.

# Trajectory (sequence space)

Show that oracle optimized using the CbAS algorithm is better than an oracle that leads you astray into unknown regions, say one that picks random sequences from the search space.

TODO: We have yet to plot the sequences iteratively. Rather, we just concatenate all the samples together from all the iterations. The benefit of (iteratively) plotting the sequences is that it shows how the algorithm picks the next set of sequences to sample, which can be beneficial to understand the dynamics (in a Monte Carlo sense).

## Random sequences

### Load sequences

In [ ]:
from examples.gb1 import data

In [ ]:
# TODO: How do we not save the data since values can be quickly computed...
dataset = data.load_variants("lstm", "Fitness", rootdir="../../data/3gb1/processed/",
                             num_data=10000, vocab="aa20")

### Compute predictions

In [ ]:
all_oracles.keys(), all_gps.keys()

In [ ]:
Vaa = dataset[:]["arr_0"].long()
# Convert to onehot
V = torch.zeros(*Vaa.size(), vocab_size)
V.scatter_(2, torch.unsqueeze(Vaa, 2), 1)

# Evaluate on (g-mean) oracle to get predictions
gmean_oracle = all_oracles["g-mean"]
gmean_oracle.eval()
with torch.no_grad():
    y_pred = gmean_oracle(V)
    mu_oracle, sigma_oracle = y_pred[:,0], torch.sqrt(y_pred[:,1])
        
# Evaluate on (g-mean) gp to get GT predictions
gmean_gp = all_gps["g-mean"]
mu_gp, sigma_gp = gmean_gp.predict(Vaa, return_std=True)
mu_gp, sigma_gp = mu_gp.squeeze(), sigma_gp.squeeze()

In [ ]:
iters = 50
random_results = {
    "y_oracle": mu_oracle.view(iters, -1),
    "y_gt": mu_gp.view(iters, -1),
    "seq": np.array(["".join(tokenizer.decode(seq)) for seq in Vaa.numpy()]).reshape(50, -1)
}

In [ ]:
# What are the topk sequences and their predicted oracle/GT values?
random_y_oracle = np.array(random_results["y_oracle"]).flatten()
topk_random_idx = np.argsort(random_y_oracle)[-10:]

seq_topk_random = np.array(random_results["seq"]).flatten()[topk_random_idx]
yt_topk_random = random_y_oracle[topk_random_idx]
yt_gt_topk_random = np.array(random_results["y_gt"]).flatten()[topk_random_idx]

In [ ]:
yt_topk_random, yt_gt_topk_random, seq_topk_random

In [ ]:
quantile = 0.80
sort = True

plt.figure(figsize=(16,8))
combined_results = {"cbas": cbas_results, "random": random_results}
colors = cm.get_cmap("coolwarm")(np.linspace(0, 1, len(combined_results)))
for label, values, c in zip(combined_results.keys(), combined_results.values(), colors):
    # Compute 80th percentile range for y_oracle
    oracle_samples = np.ma.masked_invalid(values["y_oracle"])
    per = np.percentile(oracle_samples, quantile*100, axis=-1)
    per = np.reshape(per, newshape=(oracle_samples.shape[0], 1))
    oracle_idxs = np.where(oracle_samples > per)

    # Average values for samples that are greater than quantile
    oracle_vals = np.zeros_like(oracle_samples)
    oracle_vals[oracle_idxs] = oracle_samples[oracle_idxs]
    oracle_vals = oracle_vals.sum(1) / (oracle_vals != 0).sum(1)

    gt_samples = np.ma.masked_invalid(values["y_gt"])
    gt_vals = np.zeros_like(gt_samples)
    gt_vals[oracle_idxs] = gt_samples[oracle_idxs]
    gt_vals = gt_vals.sum(1) / (gt_vals != 0).sum(1)

    # Sort oracle values (ascending) when plotting
    if sort:
        oracle_idxs = np.argsort(oracle_vals)
        oracle_vals = oracle_vals[oracle_idxs]
        gt_vals = gt_vals[oracle_idxs]

    # Plot values
    plt.plot(np.arange(1, len(oracle_vals)+1), oracle_vals, color=c, linestyle="dashed")
    plt.plot(np.arange(1, len(gt_vals)+1), gt_vals, color=c, label=label)
plt.xlabel("Timestep ($t$)")
plt.ylabel(f"${quantile * 100:.0f}\%$ Fitness ($y$)")
plt.legend()
plt.show()

### Normal projection

In [ ]:
pos = [38, 39, 40, 53]
char_seqs = ["".join(tokenizer.decode(seq)) for seq in Vaa[:, pos].numpy()]

In [ ]:
combs = generator.gen(n=2)
fitness_df = pd.DataFrame(0.0, index=combs, columns=combs)
# Place fitness values into df
for variant, fitness in zip(char_seqs, mu_oracle):
    x, y = variant[:2], variant[2:]
    fitness_df.at[x,y] = fitness
fitness_df.head()

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(fitness_df, cmap='Blues', square=True)

### Hilbert curve projection

In [ ]:
def hilbert(order):
    U = (-1, 0)
    R = (0, 1)
    D = (1, 0)
    L = (0, -1)

    URDR = (U, R, D, R)
    RULU = (R, U, L, U)
    URDD = (U, R, D, D)
    LDRR = (L, D, R, R)
    RULL = (R, U, L, L)
    DLUU = (D, L, U, U)
    LDRD = (L, D, R, D)
    DLUL = (D, L, U, L)

    inception = {
        URDR: (RULU, URDR, URDD, LDRR),
        RULU: (URDR, RULU, RULL, DLUU),
        URDD: (RULU, URDR, URDD, LDRD),
        LDRR: (DLUL, LDRD, LDRR, URDR),
        RULL: (URDR, RULU, RULL, DLUL),
        DLUU: (LDRD, DLUL, DLUU, RULU),
        LDRD: (DLUL, LDRD, LDRR, URDD),
        DLUL: (LDRD, DLUL, DLUU, RULL)
    }

    pos = [(2 ** order) - 1, 0, 0]  # y, x, linear

    def walk(steps, level):
        if level > 1:
            for substeps in inception[steps]:
                for subpos in walk(substeps, level - 1):
                    yield subpos
        else:
            for step in steps:
                yield pos
                pos[0] += step[0]  # y
                pos[1] += step[1]  # x
                pos[2] += 1  # linear

    return walk(URDR, order)

In [ ]:
order = 9
ntotal = (2**order)**2
loc = np.zeros((ntotal, 2), dtype=np.int)
for point in hilbert(order):
    loc[point[-1]] = point[:2]

In [ ]:
combs_4char = generator.gen(n=4)
seq_val_map = dict(zip(char_seqs, mu_oracle))
mask = np.array([seq_val_map.get(seq, 0) for seq in combs_4char])

In [ ]:
vals = np.zeros(ntotal)
vals[:len(mask)] = mask

# Place values at specified locations
out = np.zeros((2**order, 2**order))
for (x,y), v in zip(loc, vals):
    out[x,y] = v

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(out, cmap="Blues", square=True, mask=(out == 0))

## CbAS-generated sequences

### Load sequences

In [ ]:
with open("../../dumps/3gb1/cbas/oracle=g-mean__gp=g-mean.json", "rb") as json_file:
    results = json.load(json_file)
gmean_seqs_cbas = np.array(results["seq"])
y_cbas = np.array(results["y_oracle"])
y_gt = np.array(results["y_gt"])

### Preprocess sequences

In [ ]:
import re
# Flatten; we don't care about the iterative samples plot (for now)
seqs_cbas = gmean_seqs_cbas.flatten()
y_cbas = y_cbas.flatten()
# Remove sequences that are invalid (aka None)
valid_mask = seqs_cbas != None
valid_seqs = seqs_cbas[valid_mask]
valid_y = y_cbas[valid_mask]
# Remove sequences that don't follow WT template (except at mutated positions)
template = list("MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE")
aa20_regex = "|".join(list(tokenizer.vocab.keys()))
pos = [38, 39, 40, 53]
for idx in pos:
    template[idx] = f"[{aa20_regex}]"
regex_pattern = "".join(template)

seq_mask = np.array([True if re.findall(regex_pattern, seq, re.IGNORECASE) != [] else False
                     for seq in valid_seqs])

valid_seqs = valid_seqs[seq_mask]
valid_y = valid_y[seq_mask]
# Keep only first instance of unique seqs
_, unique_idx = np.unique(valid_seqs, return_index=True)
unique_seqs = valid_seqs[unique_idx]
unique_y = valid_y[unique_idx]
# Split each seq into individual char
unique_seqs = np.array([list(seq) for seq in unique_seqs])

unique_seqs.shape, unique_y.shape

### Normal projection

In [ ]:
unique_char_seqs = ["".join(seq) for seq in unique_seqs[:, pos]]
combs = generator.gen(n=2)
fitness_df = pd.DataFrame(0.0, index=combs, columns=combs)
# Place fitness values into df
for variant, fitness in zip(unique_char_seqs, unique_y):
    x, y = variant[:2], variant[2:]
    fitness_df.at[x,y] = fitness
fitness_df.head()

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(fitness_df, cmap='Purples', square=True)

Notice how the CbAS procedure focuses on looking at sequences that are good (based off oracle). This can be observed by the fact that certain lines are more densely sampled and "observed". Compared to random sequences, notice that the values predicted are (on average) better.

### Hilbert curve projection

In [ ]:
order = 9
ntotal = (2**order)**2
loc = np.zeros((ntotal, 2), dtype=np.int)
for point in hilbert(order):
    loc[point[-1]] = point[:2]

In [ ]:
combs_4char = generator.gen(n=4)
seq_val_map = dict(zip(["".join(seq) for seq in unique_char_seqs], unique_y))

mask = np.array([seq_val_map.get(seq, 0) for seq in combs_4char])

In [ ]:
vals = np.zeros(ntotal)
vals[:len(mask)] = mask

# Place values at specified locations
out = np.zeros((2**order, 2**order))
for (x,y), v in zip(loc, vals):
    out[x,y] = v

In [ ]:
plt.figure(figsize=(16,16))
sns.heatmap(out, cmap="Purples", square=True, mask=(out == 0))

# Random vs designed

Subset the plots by number of mutations from the WT (i.e. hamming distance). What we hope to observe, by using this approach, is that designing protein sequences (with varying amounts of point mutations), does, in fact, result in better fitness than just sampling random mutations from the search space.

NOTE: In this case, it might not be better than random because our oracle is bad at determining the fitness score between sequences. From the oracle's viewpoint, most sequences have the same fitness score (with some minor variance). Maybe the proxy (GT) is a better because it shows that the CbAS is not going into "bad" sequence search space.

## Oracle

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.kdeplot(mu_oracle.numpy(), shade=True, label="Random", color="grey")
ax = sns.kdeplot(unique_y, shade=True, label="CbAS", color="mediumseagreen")
ax.set_xlabel("Fitness ($y$)")
ax.set_title(f"All Sequences")

In [ ]:
template = list("MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE")
cbas_hamming = (unique_seqs != template).sum(axis=-1)
random_hamming = (np.array([tokenizer.decode(seq)
                            for seq in Vaa.numpy()]) != template).sum(axis=-1)
mutation_classes, random_hamming_count = np.unique(random_hamming, return_counts=True)
mutation_classes, cbas_hamming_count = np.unique(cbas_hamming, return_counts=True)

if 0 in mutation_classes:
    mutation_classes = mutation_classes[1:]
mutation_classes

In [ ]:
cols = ["Random", "CbAS", "Hamming"]

df = pd.DataFrame(columns=cols)
for t in mutation_classes:
    random_hamming_vals = mu_oracle[np.where(random_hamming == t)]
    cbas_hamming_vals = unique_y[np.where(cbas_hamming == t)]
    
    max_num_vals = max(random_hamming_vals.shape[0], cbas_hamming_vals.shape[0])
    random_hamming_class = torch.Tensor([np.nan] * max_num_vals)
    random_hamming_class[:len(random_hamming_vals)] = random_hamming_vals
    cbas_hamming_class = np.array([np.nan] * max_num_vals)
    cbas_hamming_class[:len(cbas_hamming_vals)] = cbas_hamming_vals
    hamming_class = np.array([t] * max_num_vals)
    
    class_dict = dict(zip(cols, [random_hamming_class,
                                 cbas_hamming_class,
                                 hamming_class]))
    df = df.append(pd.DataFrame(class_dict), ignore_index=True)
df.head()

In [ ]:
# Compute WT fitness
wt_aa = torch.LongTensor(tokenizer.encode(template)).unsqueeze(0)
wt = torch.zeros(*wt_aa.size(), vocab_size)
wt.scatter_(2, torch.unsqueeze(wt_aa, 2), 1)
oracle.eval()
with torch.no_grad():
    mu_wt = gmean_oracle(wt)[:, 0].squeeze().numpy()

# Global params for figure
y_min = min(df["Random"].min(), df["CbAS"].min())
y_max = max(df["Random"].max(), df["CbAS"].max()) + 0.1
wt_color = "black"
cbas_color = "mediumseagreen"
random_color = "grey"
# Compute (during iteration)
frac_above_wt = {"random": [], "cbas": []}

fig = plt.figure(figsize=(16,16))
gs = gridspec.GridSpec(nrows=4, ncols=4, figure=fig)
for idx, t in enumerate(mutation_classes):
    # Create axes
    ax = fig.add_subplot(gs[1:, idx])
    
    # Plot distribution
    class_df = df.loc[df["Hamming"] == t]
    sns.kdeplot(class_df["Random"], legend=False, vertical=True, shade=True,
                color=random_color, ax=ax)
    sns.kdeplot(class_df["CbAS"], legend=False, vertical=True, shade=True,
                color=cbas_color, ax=ax)
    
    # Set x,y-axis limits
    ax.set_xlim(0, 11)
    ax.set_ylim(y_min, y_max)
    # Make background transparent
    ax.patch.set_alpha(0)
    # Remove axis ticks, and labels
    ax.set_xticklabels([])
    ax.set_xticks([])
    if idx == 0:
        ax.set_ylabel("Fitness ($y$)", fontsize=26)
    else:
        ax.set_yticklabels([])
    # Remove borders (aka despine)
    spines = ["top", "right", "bottom"]
    for s in spines:
        ax.spines[s].set_visible(False)
        
    # Indicate location of mu_wt 
    if idx < len(mutation_classes) - 1:
        ax.axhline(y=mu_wt, c=wt_color, ls="--")
    else:
        xloc = 0.6
        yloc = (mu_wt - y_min) / (y_max - y_min)
        ax.axhline(y=mu_wt, xmax=xloc, c=wt_color, ls="--")
        ax.text(xloc + 0.05, yloc, s="WT", color=wt_color, ha="center",
                va="center", transform=ax.transAxes, fontsize=26, rotation=90)
    
    # Label num mutations (hamming distance)
    ax.text(-0.01, -0.02, s=f"${t}$", fontweight="bold", ha="left",
            va="center", transform=ax.transAxes, fontsize=22)
    
    # Label num examples in random, cbas
    total_random = (~class_df["Random"].isnull()).sum()
    total_cbas = (~class_df["CbAS"].isnull()).sum()
    ax.text(0.02, 0.02, s=f"${total_random}$", color=random_color,
            ha="left", va="center", transform=ax.transAxes, fontsize=24)
    ax.text(0.02, 0.98, s=f"${total_cbas}$", color=cbas_color,
            ha="left", va="center", transform=ax.transAxes, fontsize=24)
    if idx == len(mutation_classes) - 1:
        # The \\ in front of each word is a hack to properly align them
        ax.text(0.10, 0.15, s=r"\textbf{\\Random\\mutants}", color=random_color,
                ha="left", va="center", transform=ax.transAxes, fontsize=24)
        ax.text(0.10, 0.75, s=r"\textbf{\\Designed\\mutants}", color=cbas_color,
                ha="left", va="center", transform=ax.transAxes, fontsize=24)
        ax.text(0.3, 0.94, s=r"\textit{Number of samples}", color=cbas_color,
                ha="center", va="center", transform=ax.transAxes, fontsize=24)
    
    # Compute fraction above wt fitness
    random_frac_above_wt = (class_df["Random"] > mu_wt).sum() / total_random
    cbas_frac_above_wt = (class_df["CbAS"] > mu_wt).sum() / total_cbas
    frac_above_wt["random"].append(random_frac_above_wt)
    frac_above_wt["cbas"].append(cbas_frac_above_wt)
    
    # Center global x-axis label
    if idx == (len(mutation_classes) // 2) - 1:
        ax.set_xlabel("Mutations", labelpad=25, fontsize=26)

# Plot fraction above wt (for both )
ax1 = fig.add_subplot(gs[0, :2])
ax1.plot(frac_above_wt["random"], c=random_color, linewidth=3, label="Random")
ax1.plot(frac_above_wt["cbas"], c=cbas_color, linewidth=3, label="Designed")
formula = (r"$\frac{\left\vert \{ y_{\mathrm{oracle}} | y_{\mathrm{oracle}} > " +
           r"y_{\mathrm{wt}}\} \right\vert}{\left\vert \{y_{\mathrm{oracle}}\} \right\vert}$")
ax1.text(0.5, 0.85, s=formula, color="black", ha="center",
        va="center", transform=ax1.transAxes, fontsize=30)
# ax1.text(0.25, 0.10, s=r"\textbf{Random mutants}", color=random_color,
#         ha="left", va="center", transform=ax1.transAxes, fontsize=16)
# ax1.text(0.6, 0.58, s=r"\textbf{Designed mutants}", color=cbas_color,
#         ha="left", va="center", transform=ax1.transAxes, fontsize=16)
ax1.set_xlim(0, len(mutation_classes) - 1)
ax1.set_xticks(np.arange(0, len(mutation_classes) - 1, 1))
ax1.set_xticklabels([])
ax1.set_ylabel("Fraction above WT", fontsize=26)
ax1.legend()

fig.tight_layout()
fig.subplots_adjust(wspace=-0.5) # overlap graphs
plt.show()

Inspired by: https://sci-hub.tw/10.1126/science.aaw2900
http://www.ipam.ucla.edu/abstract/?tid=15908&pcode=LCO2020

## GP (Ground Truth)

In [ ]:
Vaa = dataset[:]["arr_0"].long()

# Evaluate on (g-mean) GP to get GT predictions
gmean_gp = all_gps["all"]
mu_gt, sigma_gt = gmean_gp.predict(Vaa, return_std=True)
mu_gt, sigma_gt = mu_gt.squeeze().numpy(), sigma_gt.squeeze().numpy()

In [ ]:
# Flatten; we don't care about the iterative samples plot (for now)
seqs_cbas = gmean_seqs_cbas.flatten()
y_gt = y_gt.flatten()
# Remove sequences that are invalid (aka None)
valid_mask = seqs_cbas != None
valid_seqs = seqs_cbas[valid_mask]
valid_y_gt = y_gt[valid_mask]
# Remove sequences that don't follow WT template (except at mutated positions)
template = list("MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE")
aa20_regex = "|".join(list(tokenizer.vocab.keys()))
pos = [38, 39, 40, 53]
for idx in pos:
    template[idx] = f"[{aa20_regex}]"
regex_pattern = "".join(template)

seq_mask = np.array([True if re.findall(regex_pattern, seq, re.IGNORECASE) != [] else False
                     for seq in valid_seqs])

valid_seqs = valid_seqs[seq_mask]
valid_y_gt = valid_y_gt[seq_mask]
# Keep only first instance of unique seqs
_, unique_idx = np.unique(valid_seqs, return_index=True)
unique_seqs = valid_seqs[unique_idx]
unique_y_gt = valid_y_gt[unique_idx]
# Split each seq into individual char
unique_seqs = np.array([list(seq) for seq in unique_seqs])

unique_seqs.shape, unique_y_gt.shape

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.kdeplot(mu_gt, shade=True, label="Random (GT)", color="grey")
ax = sns.kdeplot(unique_y_gt, shade=True, label="CbAS (GT)", color="mediumseagreen")
ax.set_xlabel("Fitness ($y$)")
ax.set_title(f"All Sequences")

In [ ]:
template = list("MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE")
cbas_hamming = (unique_seqs != template).sum(axis=-1)
random_hamming = (np.array([tokenizer.decode(seq)
                            for seq in Vaa.numpy()]) != template).sum(axis=-1)
mutation_classes, random_hamming_count = np.unique(random_hamming, return_counts=True)
mutation_classes, cbas_hamming_count = np.unique(cbas_hamming, return_counts=True)

if 0 in mutation_classes:
    mutation_classes = mutation_classes[1:]
mutation_classes

In [ ]:
cols = ["Random", "CbAS", "Hamming"]

df = pd.DataFrame(columns=cols)
for t in mutation_classes:
    random_hamming_vals = mu_gt[np.where(random_hamming == t)]
    cbas_hamming_vals = unique_y_gt[np.where(cbas_hamming == t)]
    
    max_num_vals = max(random_hamming_vals.shape[0], cbas_hamming_vals.shape[0])
    random_hamming_class = np.array([np.nan] * max_num_vals)
    random_hamming_class[:len(random_hamming_vals)] = random_hamming_vals
    cbas_hamming_class = np.array([np.nan] * max_num_vals)
    cbas_hamming_class[:len(cbas_hamming_vals)] = cbas_hamming_vals
    hamming_class = np.array([t] * max_num_vals)
    
    class_dict = dict(zip(cols, [random_hamming_class,
                                cbas_hamming_class,
                                hamming_class]))
    df = df.append(pd.DataFrame(class_dict), ignore_index=True)
df.head()

In [ ]:
# Compute WT fitness
wt_aa = torch.LongTensor(tokenizer.encode(template)).unsqueeze(0)
mu_wt = gmean_gp.predict(wt_aa, return_std=False).item()

# Global params for figure
y_min = min(df["Random"].min(), df["CbAS"].min())
y_max = max(df["Random"].max(), df["CbAS"].max()) + 0.1
wt_color = "black"
cbas_color = "mediumseagreen" # "mediumslateblue"
random_color = "grey"
# Compute (during iteration)
frac_above_wt = {"random": [], "cbas": []}

fig = plt.figure(figsize=(16,16))
gs = gridspec.GridSpec(nrows=4, ncols=4, figure=fig)
for idx, t in enumerate(mutation_classes):
    # Create axes
    ax = fig.add_subplot(gs[1:, idx])
    
    # Plot distribution
    class_df = df.loc[df["Hamming"] == t]
    sns.kdeplot(class_df["Random"], legend=False, vertical=True, shade=True,
                color=random_color, ax=ax)
    sns.kdeplot(class_df["CbAS"], legend=False, vertical=True, shade=True,
                color=cbas_color, ax=ax)
    
    # Set x,y-axis limits
    ax.set_xlim(0, 2.5)
    ax.set_ylim(y_min, y_max)
    # Make background transparent
    ax.patch.set_alpha(0)
    # Remove axis ticks, and labels
    ax.set_xticklabels([])
    ax.set_xticks([])
    if idx == 0:
        ax.set_ylabel("Fitness ($y$)", fontsize=20)
    else:
        ax.set_yticklabels([])
    # Remove borders (aka despine)
    spines = ["top", "right", "bottom"]
    for s in spines:
        ax.spines[s].set_visible(False)
        
    # Indicate location of mu_wt 
    if idx < len(mutation_classes) - 1:
        ax.axhline(y=mu_wt, c=wt_color, ls="--")
    else:
        xloc = 0.6
        yloc = (mu_wt - y_min) / (y_max - y_min)
        ax.axhline(y=mu_wt, xmax=xloc, c=wt_color, ls="--")
        ax.text(xloc + 0.05, yloc, s="WT", color=wt_color, ha="center",
                va="center", transform=ax.transAxes, fontsize=16, rotation=90)
    
    # Label num mutations (hamming distance)
    ax.text(-0.01, -0.02, s=f"${t}$", fontweight="bold", ha="left",
            va="center", transform=ax.transAxes, fontsize=18)
    
    # Label num examples in random, cbas
    total_random = (~class_df["Random"].isnull()).sum()
    total_cbas = (~class_df["CbAS"].isnull()).sum()
    ax.text(0.02, 0.02, s=f"${total_random}$", color=random_color,
            ha="left", va="center", transform=ax.transAxes, fontsize=16)
    ax.text(0.02, 0.98, s=f"${total_cbas}$", color=cbas_color,
            ha="left", va="center", transform=ax.transAxes, fontsize=16)
    if idx == len(mutation_classes) - 1:
        ax.text(0.10, 0.15, s=r"\textbf{Random mutants}", color=random_color,
                ha="left", va="center", transform=ax.transAxes, fontsize=16)
        ax.text(0.10, 0.75, s=r"\textbf{Designed mutants}", color=cbas_color,
                ha="left", va="center", transform=ax.transAxes, fontsize=16)
        ax.text(0.3, 0.96, s=r"\textit{Number of samples}", color=cbas_color,
                ha="center", va="center", transform=ax.transAxes, fontsize=16)
    
    # Compute fraction above wt fitness
    random_frac_above_wt = (class_df["Random"] > mu_wt).sum() / total_random
    cbas_frac_above_wt = (class_df["CbAS"] > mu_wt).sum() / total_cbas
    frac_above_wt["random"].append(random_frac_above_wt)
    frac_above_wt["cbas"].append(cbas_frac_above_wt)
    
    # Center global x-axis label
    if idx == (len(mutation_classes) // 2) - 1:
        ax.set_xlabel("Mutations", labelpad=25, fontsize=20)

# Plot fraction above wt (for both )
ax1 = fig.add_subplot(gs[0, :2])
ax1.plot(frac_above_wt["random"], c=random_color, linewidth=3, label="Random")
ax1.plot(frac_above_wt["cbas"], c=cbas_color, linewidth=3, label="Designed")
# ax1.text(0.25, 0.10, s=r"\textbf{Random mutants}", color=random_color,
#         ha="left", va="center", transform=ax1.transAxes, fontsize=16)
# ax1.text(0.6, 0.58, s=r"\textbf{Designed mutants}", color=cbas_color,
#         ha="left", va="center", transform=ax1.transAxes, fontsize=16)
ax1.set_xlim(0, len(mutation_classes) - 1)
ax1.set_xticks(np.arange(0, len(mutation_classes) - 1, 1))
ax1.set_xticklabels([])
ax1.set_ylabel("Fraction above WT fitness", fontsize=20)
ax1.legend()

fig.tight_layout()
fig.subplots_adjust(wspace=-0.5) # overlap graphs
plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))

for ax, t in zip(axarr.flatten(), mutation_classes):
    cbas_hamming_mean_gt = unique_y_gt[np.where(cbas_hamming == t)]
    random_hamming_mean_gt = mu_gt[np.where(random_hamming == t)]
    sns.kdeplot(random_hamming_mean_gt, shade=True, label="Random (GT)", color="grey", ax=ax)
    sns.kdeplot(cbas_hamming_mean_gt, shade=True, label="CbAS (GT)", color="mediumseagreen", ax=ax)
    ax.set_title(f"Mutations (n=${t}$)")